In [1]:
import pandas as pd
import keras
from keras.layers import LSTM, Input, Dense, TimeDistributed, Concatenate, GRU
import keras.backend as K
from keras import regularizers, Model
import tensorflow as tf
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import csv
import pickle
import datetime
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

Using TensorFlow backend.


In [2]:
data_1992 = pd.read_excel(r'./data/1992.xlsx', sheet_name='Export_Output1992')
data_1992 = data_1992.drop('POINTID', axis=1)
data_1992 = data_1992[data_1992['DEM']>=0]
data_1992 = data_1992[data_1992['NDVI']>=0]
data_1992 = data_1992[data_1992['lu']>=0]
data_1992.loc[data_1992['gdp'] < 0, 'gdp'] = np.mean(data_1992[data_1992['gdp'] > 0]['gdp'])
data_1992.loc[data_1992['pcp'] < 0, 'pcp'] = np.mean(data_1992[data_1992['pcp'] > 0]['pcp'])
data_1992.loc[data_1992['tem'] < 0, 'tem'] = np.mean(data_1992[data_1992['tem'] > 0]['tem'])
data_1992.loc[data_1992['pop'] < 0, 'pop'] = np.mean(data_1992[data_1992['pop'] > 0]['pop'])

data_1998 = pd.read_excel(r'./data/1998.xlsx', sheet_name='Export_Output1998')
data_1998 = data_1998.drop('POINTID', axis=1)
data_1998 = data_1998[data_1998['DEM']>=0]
data_1998 = data_1998[data_1998['NDVI']>=0]
data_1998 = data_1998[data_1998['lu']>=0]
data_1998.loc[data_1998['gdp'] < 0, 'gdp'] = np.mean(data_1998[data_1998['gdp'] > 0]['gdp'])
data_1998.loc[data_1998['pcp'] < 0, 'pcp'] = np.mean(data_1998[data_1998['pcp'] > 0]['pcp'])
data_1998.loc[data_1998['tem'] < 0, 'tem'] = np.mean(data_1998[data_1998['tem'] > 0]['tem'])
data_1998.loc[data_1998['pop'] < 0, 'pop'] = np.mean(data_1998[data_1998['pop'] > 0]['pop'])

data_2007 = pd.read_excel(r'./data/2007.xlsx', sheet_name='Export_Output2007')
data_2007 = data_2007.drop('POINTID', axis=1)
data_2007 = data_2007[data_2007['NDVI']>=0]
data_2007 = data_2007[data_2007['DEM']>=0]
data_2007 = data_2007[data_2007['lu']>=0]
data_2007.loc[data_2007['gdp'] < 0, 'gdp'] = np.mean(data_2007[data_2007['gdp'] > 0]['gdp'])
data_2007.loc[data_2007['pcp'] < 0, 'pcp'] = np.mean(data_2007[data_2007['pcp'] > 0]['pcp'])
data_2007.loc[data_2007['tem'] < 0, 'tem'] = np.mean(data_2007[data_2007['tem'] > 0]['tem'])
data_2007.loc[data_2007['pop'] < 0, 'pop'] = np.mean(data_2007[data_2007['pop'] > 0]['pop'])


data_2018 = pd.read_excel(r'./data/2018.xlsx', sheet_name='Export_Output_2018')
data_2018 = data_2018.drop('POINTID', axis=1)
data_2018 = data_2018[data_2018['DEM']>=0]
data_2018 = data_2018[data_2018['NDVI']>=0]
data_2018 = data_2018[data_2018['lu']>=0]
data_2018.loc[data_2018['gdp'] < 0, 'gdp'] = np.mean(data_2018[data_2018['gdp'] > 0]['gdp'])
data_2018.loc[data_2018['pcp'] < 0, 'pcp'] = np.mean(data_2018[data_2018['pcp'] > 0]['pcp'])
data_2018.loc[data_2018['tem'] < 0, 'tem'] = np.mean(data_2018[data_2018['tem'] > 0]['tem'])
data_2018.loc[data_2018['pop'] < 0, 'pop'] = np.mean(data_2018[data_2018['pop'] > 0]['pop'])


In [3]:
#Original data point
data_dict1992 = dict()
data_dict1998 = dict()
data_dict2007 = dict()
data_dict2018 = dict()

for index,row in data_1992.iterrows():
    cur_id = (int(row['X']),int(row['Y']))
    data_dict1992[cur_id] = {}
    data_dict1992[cur_id]['year'] = 1992
    data_dict1992[cur_id]['x'] = cur_id[0]
    data_dict1992[cur_id]['y'] = cur_id[1]
    data_dict1992[cur_id]['height'] = row['DEM']
    data_dict1992[cur_id]['dis'] = row['MEAN_NEAR_D']
    data_dict1992[cur_id]['usage'] = row['lu']
    data_dict1992[cur_id]['gdp'] = row['gdp']
    data_dict1992[cur_id]['pcp'] = row['pcp']
    data_dict1992[cur_id]['tem'] = row['tem']
    data_dict1992[cur_id]['pop'] = row['pop']
    data_dict1992[cur_id]['ndvi'] = row['NDVI']

for index,row in data_1998.iterrows():
    cur_id = (int(row['X']),int(row['Y']))
    data_dict1998[cur_id] = {}
    data_dict1998[cur_id]['year'] = 1998
    data_dict1998[cur_id]['x'] = cur_id[0]
    data_dict1998[cur_id]['y'] = cur_id[1]
    data_dict1998[cur_id]['height'] = row['DEM']
    data_dict1998[cur_id]['dis'] = row['MIN_NEAR_D']
    data_dict1998[cur_id]['usage'] = row['lu']
    data_dict1998[cur_id]['gdp'] = row['gdp']
    data_dict1998[cur_id]['pcp'] = row['pcp']
    data_dict1998[cur_id]['tem'] = row['tem']
    data_dict1998[cur_id]['pop'] = row['pop']
    data_dict1998[cur_id]['ndvi'] = row['NDVI']

for index,row in data_2007.iterrows():
    cur_id = (int(row['X']),int(row['Y']))
    data_dict2007[cur_id] = {}
    data_dict2007[cur_id]['year'] = 2007
    data_dict2007[cur_id]['x'] = cur_id[0]
    data_dict2007[cur_id]['y'] = cur_id[1]
    data_dict2007[cur_id]['height'] = row['DEM']
    data_dict2007[cur_id]['dis'] = row['MIN_NEAR_D']
    data_dict2007[cur_id]['usage'] = row['lu']
    data_dict2007[cur_id]['gdp'] = row['gdp']
    data_dict2007[cur_id]['pcp'] = row['pcp']
    data_dict2007[cur_id]['tem'] = row['tem']
    data_dict2007[cur_id]['pop'] = row['pop']
    data_dict2007[cur_id]['ndvi'] = row['NDVI']

for index,row in data_2018.iterrows():
    cur_id = (int(row['X']),int(row['Y']))
    data_dict2018[cur_id] = {}
    data_dict2018[cur_id]['year'] = 2018
    data_dict2018[cur_id]['x'] = cur_id[0]
    data_dict2018[cur_id]['y'] = cur_id[1]
    data_dict2018[cur_id]['height'] = row['DEM']
    data_dict2018[cur_id]['dis'] = row['MIN_NEAR_D']
    data_dict2018[cur_id]['usage'] = row['lu']
    data_dict2018[cur_id]['gdp'] = row['gdp']
    data_dict2018[cur_id]['pcp'] = row['pcp']
    data_dict2018[cur_id]['tem'] = row['tem']
    data_dict2018[cur_id]['pop'] = row['pop']
    data_dict2018[cur_id]['ndvi'] = row['NDVI']

In [4]:
#Merge data
data_dict = dict()
for each_id in data_dict1992:
    data_dict[each_id] = {}
    data_dict[each_id] = {}
    data_dict[each_id]['year'] = [1992]
    data_dict[each_id]['x'] = each_id[0]
    data_dict[each_id]['y'] = each_id[1]
    data_dict[each_id]['height'] = data_dict1992[each_id]['height']
    data_dict[each_id]['dis'] = [data_dict1992[each_id]['dis']]
    data_dict[each_id]['ndvi'] = [data_dict1992[each_id]['ndvi']]
    data_dict[each_id]['usage'] = [data_dict1992[each_id]['usage']]
    data_dict[each_id]['gdp'] = [data_dict1992[each_id]['gdp']]
    data_dict[each_id]['pcp'] = [data_dict1992[each_id]['pcp']]
    data_dict[each_id]['tem'] = [data_dict1992[each_id]['tem']]
    data_dict[each_id]['pop'] = [data_dict1992[each_id]['pop']]
    
for each_id in data_dict1998:
    if each_id in data_dict:
        data_dict[each_id]['year'].append(1998)
        data_dict[each_id]['dis'] .append(data_dict1998[each_id]['dis'])
        data_dict[each_id]['ndvi'].append(data_dict1998[each_id]['ndvi'])
        data_dict[each_id]['usage'].append(data_dict1998[each_id]['usage'])
        data_dict[each_id]['gdp'].append(data_dict1998[each_id]['gdp'])
        data_dict[each_id]['pcp'].append(data_dict1998[each_id]['pcp'])
        data_dict[each_id]['tem'].append(data_dict1998[each_id]['tem'])
        data_dict[each_id]['pop'].append(data_dict1998[each_id]['pop'])
    else:
        data_dict[each_id] = {}
        data_dict[each_id] = {}
        data_dict[each_id]['year'] = [1998]
        data_dict[each_id]['x'] = each_id[0]
        data_dict[each_id]['y'] = each_id[1]
        data_dict[each_id]['height'] = data_dict1998[each_id]['height']
        data_dict[each_id]['dis'] = [data_dict1998[each_id]['dis']]
        data_dict[each_id]['ndvi'] = [data_dict1998[each_id]['ndvi']]
        data_dict[each_id]['usage'] = [data_dict1998[each_id]['usage']]
        data_dict[each_id]['gdp'] = [data_dict1998[each_id]['gdp']]
        data_dict[each_id]['pcp'] = [data_dict1998[each_id]['pcp']]
        data_dict[each_id]['tem'] = [data_dict1998[each_id]['tem']]
        data_dict[each_id]['pop'] = [data_dict1998[each_id]['pop']]
    
for each_id in data_dict2007:
    if each_id in data_dict:
        data_dict[each_id]['year'].append(2007)
        data_dict[each_id]['dis'] .append(data_dict2007[each_id]['dis'])
        data_dict[each_id]['ndvi'].append(data_dict2007[each_id]['ndvi'])
        data_dict[each_id]['usage'].append(data_dict2007[each_id]['usage'])
        data_dict[each_id]['gdp'].append(data_dict2007[each_id]['gdp'])
        data_dict[each_id]['pcp'].append(data_dict2007[each_id]['pcp'])
        data_dict[each_id]['tem'].append(data_dict2007[each_id]['tem'])
        data_dict[each_id]['pop'].append(data_dict2007[each_id]['pop'])
    else:
        data_dict[each_id] = {}
        data_dict[each_id] = {}
        data_dict[each_id]['year'] = [2007]
        data_dict[each_id]['x'] = each_id[0]
        data_dict[each_id]['y'] = each_id[1]
        data_dict[each_id]['height'] = data_dict2007[each_id]['height']
        data_dict[each_id]['dis'] = [data_dict2007[each_id]['dis']]
        data_dict[each_id]['ndvi'] = [data_dict2007[each_id]['ndvi']]
        data_dict[each_id]['usage'] = [data_dict2007[each_id]['usage']]
        data_dict[each_id]['gdp'] = [data_dict2007[each_id]['gdp']]
        data_dict[each_id]['pcp'] = [data_dict2007[each_id]['pcp']]
        data_dict[each_id]['tem'] = [data_dict2007[each_id]['tem']]
        data_dict[each_id]['pop'] = [data_dict2007[each_id]['pop']]
        
for each_id in data_dict2018:
    if each_id in data_dict:
        data_dict[each_id]['year'].append(2018)
        data_dict[each_id]['dis'] .append(data_dict2018[each_id]['dis'])
        data_dict[each_id]['ndvi'].append(data_dict2018[each_id]['ndvi'])
        data_dict[each_id]['usage'].append(data_dict2018[each_id]['usage'])
        data_dict[each_id]['gdp'].append(data_dict2018[each_id]['gdp'])
        data_dict[each_id]['pcp'].append(data_dict2018[each_id]['pcp'])
        data_dict[each_id]['tem'].append(data_dict2018[each_id]['tem'])
        data_dict[each_id]['pop'].append(data_dict2018[each_id]['pop'])
    else:
        data_dict[each_id] = {}
        data_dict[each_id] = {}
        data_dict[each_id]['year'] = [2018]
        data_dict[each_id]['x'] = each_id[0]
        data_dict[each_id]['y'] = each_id[1]
        data_dict[each_id]['height'] = data_dict2018[each_id]['height']
        data_dict[each_id]['dis'] = [data_dict2018[each_id]['dis']]
        data_dict[each_id]['ndvi'] = [data_dict2018[each_id]['ndvi']]
        data_dict[each_id]['usage'] = [data_dict2018[each_id]['usage']]
        data_dict[each_id]['gdp'] = [data_dict2018[each_id]['gdp']]
        data_dict[each_id]['pcp'] = [data_dict2018[each_id]['pcp']]
        data_dict[each_id]['tem'] = [data_dict2018[each_id]['tem']]
        data_dict[each_id]['pop'] = [data_dict2018[each_id]['pop']]

In [5]:
soil_data = pd.read_excel(r'./data/soil.xlsx', sheet_name='soil')
for index,row in soil_data.iterrows():
    cur_id = (int(row['X']),int(row['Y']))
    if cur_id not in data_dict:
        continue
    else:
        data_dict[cur_id]['soil'] = int(row['soil_1'])

In [6]:
# Generate one-hot label
soil_list = []
usage_list = []
for each_id in data_dict:
    for i in range(len(data_dict[each_id]['year'])):
        if int(data_dict[each_id]['usage'][i]) not in usage_list:
            usage_list.append(int(data_dict[each_id]['usage'][i]))
    if 'soil' not in data_dict[each_id]:
        continue
    if int(data_dict[each_id]['soil']) not in soil_list:
        soil_list.append(int(data_dict[each_id]['soil']))

In [7]:
counter_1,counter_2,counter_3,counter_4,counter_5 = 0,0,0,0,0
err_list = []
for each_id in data_dict:
    if 'soil' not in data_dict[each_id]:
        err_list.append(each_id)
        counter_5+=1
        continue
    if 0 in data_dict[each_id]['dis']:
        err_list.append(each_id)
        continue
    if len(data_dict[each_id]['year']) == 1:
        counter_1+=1
        err_list.append(each_id)
    elif len(data_dict[each_id]['year']) == 2:
        counter_2+=1
        err_list.append(each_id)
    elif len(data_dict[each_id]['year']) == 3:
        counter_3+=1
        err_list.append(each_id)
    elif len(data_dict[each_id]['year']) == 4:
        counter_4+=1
    else:
        raise(ValueError)
print(counter_1,counter_2,counter_3,counter_4,counter_5)
for each_id in err_list:
    del data_dict[each_id]

15 35 201 26677 4778


In [8]:
soil_cnt = len(soil_list)
usage_cnt = len(usage_list)

for each_id in data_dict:
    cur_onehot = np.zeros(soil_cnt)
    cur_onehot[soil_list.index(int(data_dict[each_id]['soil']))] = 1
    data_dict[each_id]['soil'] = cur_onehot
    for i in range(len(data_dict[each_id]['year'])):
        cur_onehot = np.zeros(usage_cnt)
        cur_onehot[usage_list.index(int(data_dict[each_id]['usage'][i]))] = 1
        data_dict[each_id]['usage'][i] = cur_onehot

In [9]:
dynamic = []
static = []
loc_id = []
y = []

dynamic_feature = ['ndvi','dis','gdp','pcp','tem','pop','usage']
static_feature = ['x','y','height','soil']

for each_id in data_dict:
    cur_static = []
    cur_dynamic = []
    
    for i in range(4):
        tmp = []
        for each_feature in dynamic_feature:
            if each_feature == 'usage':
                tmp += list(data_dict[each_id][each_feature][i])
            else:
                tmp.append(data_dict[each_id][each_feature][i])
        cur_dynamic.append(tmp)
    cur_dynamic = np.array(cur_dynamic)
    
    for i in range(4):
        tmp = []
        for each_feature in static_feature:
            if each_feature == 'soil':
                tmp += list(data_dict[each_id][each_feature])
            else:
                tmp.append(data_dict[each_id][each_feature])
        cur_static.append(tmp)
    cur_static = np.array(cur_static)
    
    dynamic.append(cur_dynamic)
    static.append(cur_static)
    y.append(np.expand_dims(np.array(data_dict[each_id]['ndvi'])[1:], axis=-1))
    loc_id.append(each_id)

In [10]:
dynamic_transform = []
static_transform = []
y_transform = []

for each_id in range(len(static)):
    for each_year in range(4):
        dynamic_transform.append(dynamic[each_id][each_year])
        static_transform.append(static[each_id][each_year])

dynamic_transform = np.array(dynamic_transform)
static_transform = np.array(static_transform)

dynamic_scaler = sklearn.preprocessing.StandardScaler()
static_scaler = sklearn.preprocessing.StandardScaler()
dynamic_scaler = dynamic_scaler.fit(dynamic_transform)
static_scaler = static_scaler.fit(static_transform)

for each_id in range(len(static)):
    dynamic[each_id] = dynamic_scaler.transform(dynamic[each_id])
    static[each_id] = static_scaler.transform(static[each_id])

In [11]:
dynamic = np.array(dynamic)
static = np.array(static)
y = np.array(y)

In [12]:
all_ind = list(range(len(static)))
train_ind, test_ind = train_test_split(all_ind, test_size=0.1, random_state=12345)
train_ind, valid_ind = train_test_split(train_ind, test_size=0.1, random_state=12345)

dynamic_train = []
dynamic_valid = []
dynamic_test = []
static_train = []
static_valid = []
static_test = []
label_train = []
label_valid = []
label_test = []

for ind in train_ind:
    dynamic_train.append(dynamic[ind][:-1])
    static_train.append(static[ind][:-1])
    label_train.append(y[ind])

for ind in valid_ind:
    dynamic_valid.append(dynamic[ind][:-1])
    static_valid.append(static[ind][:-1])
    label_valid.append(y[ind])

for ind in test_ind:
    dynamic_test.append(dynamic[ind])
    static_test.append(static[ind])
    label_test.append(y[ind])
    
dynamic_train = np.array(dynamic_train)
dynamic_valid = np.array(dynamic_valid)
dynamic_test = np.array(dynamic_test)
static_train = np.array(static_train)
static_valid = np.array(static_valid)
static_test = np.array(static_test)
label_train = np.array(label_train)
label_valid = np.array(label_valid)
label_test = np.array(label_test)

In [13]:
def det_coeff(y_true, y_pred):
    u = K.sum(K.square(y_true - y_pred))
    v = K.sum(K.square(y_true - K.mean(y_true)))
    return K.ones_like(v) - (u / v)

In [14]:
print(label_train.shape)
print(static_train.shape)
print(dynamic_train.shape)

(21608, 3, 1)
(21608, 3, 16)
(21608, 3, 25)


In [15]:
def create_model():
    feature_size = 25
    static_size = 16
    rnn_size = 16
    l2_regular = regularizers.l2(1e-3)
    
    input_states = Input(shape=(None, feature_size), name='input_states')
    input_static = Input(shape=(None, static_size), name='input_statics')
    
    gru = GRU(rnn_size, activation='tanh', name='gru', kernel_regularizer=l2_regular, return_sequences=True)(input_states) 
    concat_2 = Concatenate(name='concat2')([gru, input_static])
    dense = TimeDistributed(Dense(8, activation='relu', kernel_regularizer=l2_regular), name='dense')(concat_2)
    output = TimeDistributed(Dense(1, kernel_regularizer=l2_regular, activation='relu'), name='output')(dense)
    model = Model([input_states, input_static], output)
    opt = keras.optimizers.Adam(lr=1e-3)
    model.compile(loss='mean_squared_error',optimizer=opt,  metrics=['mse',det_coeff])
    return model
print(create_model().summary())
model_test = create_model()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_states (InputLayer)       (None, None, 25)     0                                            
__________________________________________________________________________________________________
gru (GRU)                       (None, None, 16)     2016        input_states[0][0]               
__________________________________________________________________________________________________
input_statics (InputLayer)      (None, None, 16)     0                                            
__________________________________________________________________________________________________
concat2 (Concatenate)           (None, None, 32)     0           gru[0][0]                        
                                     

In [69]:
epoch = 300
batch_size = 1000
save_dir = './trained_weights/model'

valid_loss = []
train_loss = []

model = create_model()
save_callback = keras.callbacks.ModelCheckpoint(save_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
history = model.fit(x=[dynamic_train, static_train], y=label_train, batch_size=batch_size, epochs=epoch, callbacks=[save_callback], verbose=1, validation_data=([dynamic_valid, static_valid],label_valid), shuffle=True)
pickle.dump(history,open(r'./trained_weights/m_history','wb'))

t_coeff: 0.6211 - val_loss: 0.0043 - val_mean_squared_error: 0.0035 - val_det_coeff: 0.6366

Epoch 00227: val_loss did not improve from 0.00427
Epoch 228/300
21608/21608 [==============================] - 0s 8us/step - loss: 0.0043 - mean_squared_error: 0.0036 - det_coeff: 0.6222 - val_loss: 0.0043 - val_mean_squared_error: 0.0035 - val_det_coeff: 0.6390

Epoch 00228: val_loss improved from 0.00427 to 0.00427, saving model to ./trained_weights/model
Epoch 229/300
21608/21608 [==============================] - 0s 10us/step - loss: 0.0043 - mean_squared_error: 0.0036 - det_coeff: 0.6214 - val_loss: 0.0043 - val_mean_squared_error: 0.0035 - val_det_coeff: 0.6368

Epoch 00229: val_loss did not improve from 0.00427
Epoch 230/300
21608/21608 [==============================] - 0s 7us/step - loss: 0.0043 - mean_squared_error: 0.0036 - det_coeff: 0.6219 - val_loss: 0.0043 - val_mean_squared_error: 0.0035 - val_det_coeff: 0.6392

Epoch 00230: val_loss improved from 0.00427 to 0.00426, saving mod

In [16]:
save_dir = './trained_weights/'
model_test.load_weights(save_dir+'model')

In [17]:
y_pred = []
y_true = []

y_pred = model_test.predict([dynamic_test[:, :-1, :], static_test[:, :-1, :]])
y_true = label_test

y_pred = y_pred.flatten()
y_true = y_true.flatten()

In [18]:
print('Mean Squared Error: %.6f'%sklearn.metrics.mean_squared_error(y_pred=y_pred, y_true=y_true))
print('Mean Absolute Error: %.6f'%sklearn.metrics.mean_absolute_error(y_pred=y_pred, y_true=y_true))
print('R2 Score: %.6f'%sklearn.metrics.r2_score(y_pred=y_pred, y_true=y_true))
adj_r2 = 1-(1-sklearn.metrics.r2_score(y_pred=y_pred, y_true=y_true))*(len(y_pred)-1)/(len(y_pred)-5-1)
print('Adjusted R2 Score: %.6f'%adj_r2)

Mean Squared Error: 0.003523
Mean Absolute Error: 0.031801
R2 Score: 0.613744
Adjusted R2 Score: 0.613502
